In [90]:
import plotly.graph_objects as go
import pandas as pd

In [91]:


df = pd.read_csv(r'data/activity.csv', encoding='utf-8',sep = ",", header=0, skiprows=[1])
df


,Duration,Distance,OriginalPace,HeartRate,Cadence,PowerOriginal,CalculatedPace,CalculatedStrideLength,CalculatedAerobicEfficiencyPace,CalculatedAerobicEfficiencyPower,CalculatedEfficiencyIndex,Unnamed: 11
0,1,0.00000,NaN,94,0,0,NaN,0,NaN,NaN,NaN,NaN
1,1,0.00000,NaN,94,0,0,NaN,0,NaN,NaN,NaN,NaN
2,1,0.00000,NaN,96,0,0,NaN,0,NaN,NaN,NaN,NaN
3,1,0.00000,NaN,96,0,0,NaN,0,NaN,NaN,NaN,NaN
4,1,0.00000,NaN,97,0,0,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1798,1,11.94671,319.0,132,51,32,NaN,184,1.12,0.24,5.878,NaN
1799,1,11.95022,337.0,131,40,18,NaN,223,1.06,0.14,9.891,NaN
1800,1,11.95264,365.0,130,30,5,NaN,274,0.99,0.04,32.877,NaN
1801,1,11.95449,384.0,130,22,0,NaN,355,0.94,NaN,NaN,NaN


In [92]:
power_data = df['PowerOriginal'][10:300] 

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=power_data.index,
    y=power_data,
    mode='lines+markers',
    name='Power'
))






In [93]:
# Auflösung 10W
resolution = 200

last_index = 0
series_list = []

max_power = power_data.max()

power_data_cut = power_data[power_data>max_power-resolution]

for index, value in power_data_cut.items():
    if last_index and last_index == index - 1:
        series_list[-1] += 1
    else:
        series_list.append(1)
    last_index = index

print(series_list)


[30, 13, 4, 3]


In [94]:
def maxtime_spend_in_power(power_series, power):
    last_index = 0
    series_list = [0]

    max_power = power_data.max()
    power_data_cut = power_data[power_data>max_power-power]

    for index, value in power_data_cut.items():
        if last_index and last_index == index - 1:
            series_list[-1] += 1
        else:
            series_list.append(1)
        last_index = index

    return max(series_list)


resolution = 10
data = []
for p in range(0, power_data.max(), resolution):
    data.append([power_data.max()-p, maxtime_spend_in_power(power_data, p)])
    #print(f"Max time spend in {power_data.max()-p}W: {maxtime_spend_in_power(power_data, p)}")

df = pd.DataFrame(data, columns=['Leistung', 'Maximale Zeit'])
df

,Leistung,Maximale Zeit
0,417,0
1,407,1
2,397,1
3,387,2
4,377,2
5,367,2
6,357,3
7,347,3
8,337,3
9,327,4


In [95]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=df["Leistung"],
    x=df["Maximale Zeit"],
    mode='lines+markers',
    name='Power Curve'
))